In [ ]:
#import modules and functions
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as snsb
#from scipy.stats import skew, kurtosis
from sklearn.preprocessing import MinMaxScaler
import csv
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from matplotlib.patches import Patch
from matplotlib.patches import Circle

load database

In [ ]:
pdb = pd. read_csv('./core_database/destress_data_pdb.csv')
pdb.columns = ['design_name', 'file_name', 'full_sequence', 'dssp_assignment', 'composition_ALA', 'composition_CYS', 
               'composition_ASP', 'composition_GLU', 'composition_PHE', 'composition_GLY', 'composition_HIS', 
               'composition_ILE', 'composition_LYS', 'composition_LEU', 'composition_MET', 'composition_ASN', 
               'composition_PRO', 'composition_GLN', 'composition_ARG', 'composition_SER', 'composition_THR',
                'composition_VAL', 'composition_TRP', 'composition_UNK', 'composition_TYR', 'ss_prop_alpha_helix', 
                'ss_prop_beta_bridge', 'ss_prop_beta_strand', 'ss_prop_3_10_helix', 'ss_prop_pi_helix', 
                'ss_prop_hbonded_turn', 'ss_prop_bend', 'ss_prop_loop', 'hydrophobic_fitness', 'isoelectric_point', 
                'charge', 'mass', 'num_residues', 'packing_density', 'budeff_total', 'budeff_steric', 'budeff_desolvation', 
                'budeff_charge', 'evoef2_total', 'evoef2_ref_total', 'evoef2_intraR_total', 'evoef2_interS_total', 
                'evoef2_interD_total', 'dfire2_total', 'rosetta_total', 'rosetta_fa_atr', 'rosetta_fa_rep', 
                'rosetta_fa_intra_rep', 'rosetta_fa_elec', 'rosetta_fa_sol', 'rosetta_lk_ball_wtd', 'rosetta_fa_intra_sol_xover4', 
                'rosetta_hbond_lr_bb', 'rosetta_hbond_sr_bb', 'rosetta_hbond_bb_sc', 'rosetta_hbond_sc', 'rosetta_dslf_fa13', 
                'rosetta_rama_prepro', 'rosetta_p_aa_pp', 'rosetta_fa_dun', 'rosetta_omega', 'rosetta_pro_close', 
                'rosetta_yhh_planarity', 'aggrescan3d_total_value', 'aggrescan3d_avg_value', 'aggrescan3d_min_value', 
                'aggrescan3d_max_value']

af2 = pd. read_csv('./core_database/destress_data_af2.csv')
af2.columns = ['design_name', 'file_name', 'full_sequence', 'dssp_assignment', 'composition_ALA', 'composition_CYS', 
               'composition_ASP', 'composition_GLU', 'composition_PHE', 'composition_GLY', 'composition_HIS', 
               'composition_ILE', 'composition_LYS', 'composition_LEU', 'composition_MET', 'composition_ASN', 
               'composition_PRO', 'composition_GLN', 'composition_ARG', 'composition_SER', 'composition_THR',
                'composition_VAL', 'composition_TRP', 'composition_UNK', 'composition_TYR', 'ss_prop_alpha_helix', 
                'ss_prop_beta_bridge', 'ss_prop_beta_strand', 'ss_prop_3_10_helix', 'ss_prop_pi_helix', 
                'ss_prop_hbonded_turn', 'ss_prop_bend', 'ss_prop_loop', 'hydrophobic_fitness', 'isoelectric_point', 
                'charge', 'mass', 'num_residues', 'packing_density', 'budeff_total', 'budeff_steric', 'budeff_desolvation', 
                'budeff_charge', 'evoef2_total', 'evoef2_ref_total', 'evoef2_intraR_total', 'evoef2_interS_total', 
                'evoef2_interD_total', 'dfire2_total', 'rosetta_total', 'rosetta_fa_atr', 'rosetta_fa_rep', 
                'rosetta_fa_intra_rep', 'rosetta_fa_elec', 'rosetta_fa_sol', 'rosetta_lk_ball_wtd', 'rosetta_fa_intra_sol_xover4', 
                'rosetta_hbond_lr_bb', 'rosetta_hbond_sr_bb', 'rosetta_hbond_bb_sc', 'rosetta_hbond_sc', 'rosetta_dslf_fa13', 
                'rosetta_rama_prepro', 'rosetta_p_aa_pp', 'rosetta_fa_dun', 'rosetta_omega', 'rosetta_pro_close', 
                'rosetta_yhh_planarity', 'aggrescan3d_total_value', 'aggrescan3d_avg_value', 'aggrescan3d_min_value', 
                'aggrescan3d_max_value']

pisces = pd. read_csv('./core_database/cullpdb_pc95.0_res0.0-3.0_len40-10000_R0.3_Xray_d2023_02_13_chains47410.csv')
pisces.columns = ["PDBchain", "len", "method", "resol", "rfac", "freerfac"]

drop NaN and duplicates

In [ ]:
pdb = pdb.drop_duplicates(subset='full_sequence', keep='first')
pdb.dropna(subset=['full_sequence'], inplace=True)

pdb= pdb.drop(['dfire2_total','budeff_total', 'budeff_steric', 'budeff_desolvation', 'budeff_charge',
               'composition_UNK','rosetta_yhh_planarity','evoef2_interD_total'],axis=1)

pdb= pdb.dropna(how='any')
pdb.reset_index(drop=True, inplace=True)

af2 = af2.drop_duplicates(subset='full_sequence', keep='first')
af2.dropna(subset=['full_sequence',], inplace=True)

af2= af2.drop(['dfire2_total','budeff_total', 'budeff_steric', 'budeff_desolvation', 'budeff_charge',
               'composition_UNK','rosetta_yhh_planarity','evoef2_interD_total'],axis=1)

af2= af2.dropna(how='any')
af2.reset_index(drop=True, inplace=True)

cluster the compositions into label form

In [ ]:

# Cluster the data about the amino acid composition into subgroups
pdb['composition_hydrophobic'] = pdb['composition_ALA'] + pdb['composition_VAL'] + pdb['composition_LEU'] + pdb['composition_ILE'] + pdb['composition_MET'] + pdb['composition_PHE'] + pdb['composition_TYR'] + pdb['composition_TRP']
pdb['composition_polar'] = pdb['composition_ASN'] + pdb['composition_GLN'] + pdb['composition_SER'] + pdb['composition_THR']
pdb['composition_acidic'] = pdb['composition_ASP'] + pdb['composition_GLU']
pdb['composition_basic'] = pdb['composition_ARG'] + pdb['composition_HIS'] + pdb['composition_LYS']
pdb['composition_cysteine'] = pdb['composition_CYS']
pdb['composition_glycine'] = pdb['composition_GLY']
pdb['composition_proline'] = pdb['composition_PRO']
pdb.drop(columns=['composition_ALA', 'composition_CYS', 'composition_ASP', 'composition_GLU', 
                'composition_PHE', 'composition_GLY', 'composition_HIS', 'composition_ILE', 
                'composition_LYS', 'composition_LEU', 'composition_MET', 'composition_ASN', 
                'composition_PRO', 'composition_GLN', 'composition_ARG', 'composition_SER', 
                'composition_THR', 'composition_VAL', 'composition_TRP', 'composition_TYR'], inplace=True)

af2['composition_hydrophobic'] = af2['composition_ALA'] + af2['composition_VAL'] + af2['composition_LEU'] + af2['composition_ILE'] + af2['composition_MET'] + af2['composition_PHE'] + af2['composition_TYR'] + af2['composition_TRP']
af2['composition_polar'] = af2['composition_ASN'] + af2['composition_GLN'] + af2['composition_SER'] + af2['composition_THR']
af2['composition_acidic'] = af2['composition_ASP'] + af2['composition_GLU']
af2['composition_basic'] = af2['composition_ARG'] + af2['composition_HIS'] + af2['composition_LYS']
af2['composition_cysteine'] = af2['composition_CYS']
af2['composition_glycine'] = af2['composition_GLY']
af2['composition_proline'] = af2['composition_PRO']
af2.drop(columns=['composition_ALA', 'composition_CYS', 'composition_ASP', 'composition_GLU', 
                'composition_PHE', 'composition_GLY', 'composition_HIS', 'composition_ILE', 
                'composition_LYS', 'composition_LEU', 'composition_MET', 'composition_ASN', 
                'composition_PRO', 'composition_GLN', 'composition_ARG', 'composition_SER', 
                'composition_THR', 'composition_VAL', 'composition_TRP', 'composition_TYR'], inplace=True)


create labels for the database

In [ ]:
#generate a list of the culling protein id PDBchain_list
PDBchain = pisces['PDBchain'].str.extract(r'(\w{4})').squeeze()
PDBchain_list = PDBchain.tolist()

pdb['Entry'] = pdb['design_name'].str[-4:].str.upper()
af2['Entry'] = af2['design_name'].apply(lambda x: x.split('-')[1])

pdb['label_1']='PDB'
af2['label_1']='AF2'
af2['label_2']=''
pdb['label_2']=''

mask = pdb['Entry'].isin(PDBchain_list)

# Assign the value 'Pisces' to the 'label' column for rows where the mask is True
pdb.loc[mask, 'label_2'] = 'Pisces'

In [ ]:
#Prepare the AF2 cath_data
cath_data_af2 = pd.read_csv ('/Users/Zachary/Desktop/Dissertation/CATH/cath-v4_3_0.alphafold-v2.2022-11-22.tsv',sep='\t+',index_col=0)
cath_data_af2.drop(columns = ['source','length','class','assignment',
                            'pLDDT','LUR','SSEs','perc_not_in_SS',
                            'packing'],inplace = True)

cath_data_af2.reset_index(inplace=True)
cath_data_af2['ID'] = cath_data_af2['domain_ID'].apply(lambda x: x.split('_')[1])
cath_data_af2.drop(columns= ['domain_ID'],inplace=True)

#Prepare the PDB CATH_data
# Read the text file into a pandas DataFrame
cath_data_pdb = pd.read_csv('/Users/Zachary/Desktop/Dissertation/CATH/CATH_daily_snapshot/cath-b-newest-all.txt', sep=' ',header = 0)
# Assign column names to the DataFrame
cath_data_pdb.columns = ["CATH domain name", 'version',"CATH num",'frag:chain']
cath_data_pdb.drop(columns= ['version','frag:chain'], inplace =True)
cath_data_pdb['ID'] = cath_data_pdb['CATH domain name'].str[:-3]

cathnum_unique = ~cath_data_pdb.copy().duplicated(subset = 'CATH num',keep = False)
unique_cath = cath_data_pdb[cathnum_unique]
filtered_cath = unique_cath.drop_duplicates(subset = 'ID',keep = 'first')

#将filter_cath从cath中删除后，剩下的行即便有着重复的PDBID，但他们的PDB ID也相同 (去掉了2462行)
filtered_pdbid = filtered_cath['ID'].unique()
rows_to_keep = ~cath_data_pdb['ID'].isin(filtered_pdbid)
cath_data_pdb = cath_data_pdb[rows_to_keep]

#在此之后，我们就可以去掉重复的PDB ID，将两个dataframe合并 （现在有139436行）
cath_data_pdb.drop_duplicates(subset = 'ID',keep = 'first',inplace = True)
cath_data_pdb.reset_index(drop = True, inplace = True)
cath_data_pdb.drop(columns= ['CATH domain name'], inplace =True)
cath_data_pdb['ID'] = cath_data_pdb['ID'].str.upper()

In [ ]:
# Merge wholedata_name with cath_data_af2
pdb['label_3']=''
af2['label_3']=''

cath_data_af2_unique = cath_data_af2.drop_duplicates(subset='ID')
cath_data_pdb_unique = cath_data_pdb.drop_duplicates(subset='ID')

af2 = af2.merge(cath_data_af2_unique[['ID', 'sfam_id']],
                                                 left_on='Entry', right_on='ID', how='left')

pdb =  pdb.merge(cath_data_pdb_unique[['ID', 'CATH num']],
                                left_on='Entry', right_on='ID', how='left' )

                            
af2['label_3']=af2['sfam_id']
pdb['label_3']=pdb['CATH num']

af2.drop (columns = ['ID','sfam_id'], inplace = True)

pdb.drop(columns=['ID','CATH num'],inplace =True)

Normailization by dividing the energy functions by number of residues

In [ ]:
#normalize the value of energy function by dividing the number of residue
pdb.loc[:,[
    'hydrophobic_fitness', 'evoef2_total', 'evoef2_ref_total', 'evoef2_intraR_total', 'evoef2_interS_total', 'rosetta_total', 'rosetta_fa_atr', 'rosetta_fa_rep', 'rosetta_fa_intra_rep', 'rosetta_fa_elec', 'rosetta_fa_sol', 'rosetta_lk_ball_wtd', 'rosetta_fa_intra_sol_xover4', 'rosetta_hbond_lr_bb', 'rosetta_hbond_sr_bb', 'rosetta_hbond_bb_sc', 'rosetta_hbond_sc', 'rosetta_dslf_fa13', 'rosetta_rama_prepro', 'rosetta_p_aa_pp', 'rosetta_fa_dun', 'rosetta_omega', 'rosetta_pro_close',  'aggrescan3d_total_value', 'aggrescan3d_avg_value', 'aggrescan3d_min_value', 'aggrescan3d_max_value',
],] = pdb.loc[:,[
    'hydrophobic_fitness', 'evoef2_total', 'evoef2_ref_total', 'evoef2_intraR_total', 'evoef2_interS_total', 'rosetta_total', 'rosetta_fa_atr', 'rosetta_fa_rep', 'rosetta_fa_intra_rep', 'rosetta_fa_elec', 'rosetta_fa_sol', 'rosetta_lk_ball_wtd', 'rosetta_fa_intra_sol_xover4', 'rosetta_hbond_lr_bb', 'rosetta_hbond_sr_bb', 'rosetta_hbond_bb_sc', 'rosetta_hbond_sc', 'rosetta_dslf_fa13', 'rosetta_rama_prepro', 'rosetta_p_aa_pp', 'rosetta_fa_dun', 'rosetta_omega', 'rosetta_pro_close',  'aggrescan3d_total_value', 'aggrescan3d_avg_value', 'aggrescan3d_min_value', 'aggrescan3d_max_value',
],].div(pdb['num_residues'],axis=0) 

af2.loc[:,[
    'hydrophobic_fitness', 'evoef2_total', 'evoef2_ref_total', 'evoef2_intraR_total', 'evoef2_interS_total', 'rosetta_total', 'rosetta_fa_atr', 'rosetta_fa_rep', 'rosetta_fa_intra_rep', 'rosetta_fa_elec', 'rosetta_fa_sol', 'rosetta_lk_ball_wtd', 'rosetta_fa_intra_sol_xover4', 'rosetta_hbond_lr_bb', 'rosetta_hbond_sr_bb', 'rosetta_hbond_bb_sc', 'rosetta_hbond_sc', 'rosetta_dslf_fa13', 'rosetta_rama_prepro', 'rosetta_p_aa_pp', 'rosetta_fa_dun', 'rosetta_omega', 'rosetta_pro_close', 'aggrescan3d_total_value', 'aggrescan3d_avg_value', 'aggrescan3d_min_value', 'aggrescan3d_max_value',
],] = af2.loc[:,[
    'hydrophobic_fitness', 'evoef2_total', 'evoef2_ref_total', 'evoef2_intraR_total', 'evoef2_interS_total',  'rosetta_total', 'rosetta_fa_atr', 'rosetta_fa_rep', 'rosetta_fa_intra_rep', 'rosetta_fa_elec', 'rosetta_fa_sol', 'rosetta_lk_ball_wtd', 'rosetta_fa_intra_sol_xover4', 'rosetta_hbond_lr_bb', 'rosetta_hbond_sr_bb', 'rosetta_hbond_bb_sc', 'rosetta_hbond_sc', 'rosetta_dslf_fa13', 'rosetta_rama_prepro', 'rosetta_p_aa_pp', 'rosetta_fa_dun', 'rosetta_omega', 'rosetta_pro_close',  'aggrescan3d_total_value', 'aggrescan3d_avg_value', 'aggrescan3d_min_value', 'aggrescan3d_max_value',
],].div(af2['num_residues'],axis=0) 

drop weird distribution

In [ ]:
#remove composition and secondary structures with weird distribution
pdb.drop(['ss_prop_alpha_helix', 'ss_prop_beta_bridge', 'ss_prop_beta_strand', 'ss_prop_3_10_helix',
           'ss_prop_pi_helix'],axis = 1, inplace = True)
af2.drop(['ss_prop_alpha_helix', 'ss_prop_beta_bridge', 'ss_prop_beta_strand', 'ss_prop_3_10_helix',
           'ss_prop_pi_helix'],axis = 1, inplace = True)


make numeric dataframe

In [ ]:
wholedata = pd.concat([pdb, af2], axis=0)#0-130665 rows - PDB #130666 - 692709 rows - AF2
wholedata.reset_index(drop=True, inplace=True)
wholedata.to_csv ('./core_database/destress_wholedata_labelled_as_rd.csv', index = False)

CORRELATION starts from here

In [ ]:
wholedata = pd.read_csv('./core_database/destress_wholedata_labelled_as_rd.csv')
wholedata.drop([
    'evoef2_ref_total','rosetta_dslf_fa13', #too many zeros
    'aggrescan3d_total_value' #have a averaged alternative
                ], axis=1, inplace=True)
#isolate the numeric data from dataframe
wholedata_num = wholedata.select_dtypes(include=[pd.np.number])

# Normalize the data
scaler = MinMaxScaler()
wholedata_normalize = pd.DataFrame(scaler.fit_transform(wholedata_num), columns=wholedata_num.columns)

calculate both Pearson's r and Spearman's rho for wholedata_normalized

In [ ]:
wholedata_normalized = wholedata_normalize.copy()
wholedata_normalized = pd.DataFrame(scaler.fit_transform(wholedata_num), columns=wholedata_num.columns)
wholedata_normalized.drop([
    'evoef2_ref_total','rosetta_dslf_fa13', #too many zeros
    'aggrescan3d_total_value', #have a averaged alternative
    'mass','num_residues','charge',#highly correlated
    'evoef2_total','evoef2_intraR_total','evoef2_interS_total',
    'rosetta_total','rosetta_fa_atr','rosetta_fa_elec','rosetta_fa_sol','rosetta_rama_prepro','rosetta_omega',
    'rosetta_p_aa_pp','rosetta_fa_dun','rosetta_hbond_bb_sc',
    'packing_density',
    'aggrescan3d_min_value'], axis=1, inplace=True)


In [ ]:
# Calculate Pearson's correlation
#pearson_corr = wholedata_normalized.corr(method='pearson')
#pearson_corr.to_csv('./output_tables/Correlation/wholedata__pearson_correlation_matrix.csv')

# Calculate Spearman's correlation
spearman_corr = wholedata_normalize.corr(method='spearman')
#spearman_corr.to_csv('./output_tables/Correlation/wholedata__spearman_correlation_matrix.csv')

find out highly correlated indexes

In [ ]:
# Define the correlation threshold
threshold = 0.01

# Find indexes with high Pearson correlation
#high_pearson_corr = np.where(np.abs(pearson_corr) > threshold)
#pearson_pairs = [(pearson_corr.index[x], pearson_corr.columns[y]) for x, y in zip(*high_pearson_corr) if x != y]

# Find indexes with high Spearman correlation
high_spearman_corr = np.where(np.abs(spearman_corr) > threshold)
spearman_pairs = [(spearman_corr.index[x], spearman_corr.columns[y]) for x, y in zip(*high_spearman_corr) if x != y]


In [ ]:
# Print the results
#print("Highly correlated pairs (Pearson):")
#for pair in pearson_pairs:
 #   print(pair, pearson_corr.loc[pair])

print("\nHighly correlated pairs (Spearman):")
for pair in spearman_pairs:
    print(pair, spearman_corr.loc[pair])

In [ ]:

threshold = 0.01
spearman_corr_high = spearman_corr.copy()
spearman_corr_high[np.abs(spearman_corr_high) < threshold] = 0
np.fill_diagonal(spearman_corr_high.values, 0)
spearman_corr_high = spearman_corr_high.loc[(spearman_corr_high != 0).any(axis=1), (spearman_corr_high != 0).any(axis=0)]

# Create a mask to show only the upper half of the heatmap
#mask = np.zeros_like(spearman_corr_high)
#mask[np.triu_indices_from(mask)] = True

mask_upper_triangle = np.triu(np.ones_like(spearman_corr_high, dtype=bool), k=1)

annot_values = spearman_corr_high.where(~mask_upper_triangle).applymap(lambda x: '{:.2f}'.format(x) if x != 0 else '')
#annot_values = spearman_corr_high.applymap(lambda x: '{:.2f}'.format(x) if x != 0 else '')

fig, ax = plt.subplots(figsize=(20, 20))
snsb.heatmap(spearman_corr_high, annot=annot_values, cmap='coolwarm', vmin=-1, vmax=1, mask=mask_upper_triangle,fmt='',annot_kws={"fontsize": 8})

plt.title("Spearman Correlation Heatmap", fontsize=30)
plt.xticks(fontsize=17)
plt.yticks(fontsize=17)
# Adjust the position of the heatmap within the figure
bottom_margin = 0.18  # Change this value to adjust the bottom margin
top_margin = 0.93 # Change this value to adjust the top margin
left_margin = 0.18  # Change this value to adjust the left margin
right_margin =0.93   # Change this value to adjust the right margin
plt.subplots_adjust(bottom=bottom_margin, top=top_margin, left=left_margin, right=right_margin)

#plt.savefig('./figures/correlation_figure_spearman_half', dpi=400)
plt.show()



In [ ]:
from collections import defaultdict

# Initialize the count dictionaries
pearson_pairs_count = defaultdict(int)
spearman_pairs_count = defaultdict(int)


# Count the frequency of each index in overlapping_pairs
for pair in pearson_pairs:
    pearson_pairs_count[pair[0]] += 1
    pearson_pairs_count[pair[1]] += 1

# Count the frequency of each index in unique_pearson_pairs
for pair in spearman_pairs:
    spearman_pairs_count[pair[0]] += 1
    spearman_pairs_count[pair[1]] += 1


# Print the results
print("Index frequency in pearson pairs:")
for index, count in pearson_pairs_count.items():
    print(f"{index}: {count}")

print("\nIndex frequency in spearman pairs:")
for index, count in spearman_pairs_count.items():
    print(f"{index}: {count}")


In [ ]:
# Convert the lists of pairs to sets
pearson_pairs_set = set(pearson_pairs)
spearman_pairs_set = set(spearman_pairs)

# Find overlapping pairs
overlapping_pairs = pearson_pairs_set.intersection(spearman_pairs_set)
print("Overlapping pairs:")
for pair in overlapping_pairs:
    print(pair)

# Find pairs present only in Pearson's correlation results
unique_pearson_pairs = pearson_pairs_set.difference(spearman_pairs_set)
print("\nUnique pairs in Pearson's correlation:")
for pair in unique_pearson_pairs:
    print(pair)

# Find pairs present only in Spearman's correlation results
unique_spearman_pairs = spearman_pairs_set.difference(pearson_pairs_set)
print("\nUnique pairs in Spearman's correlation:")
for pair in unique_spearman_pairs:
    print(pair)
    

remove sequence properties

In [ ]:
wholedata_rd =  wholedata_normalized[[
'ss_prop_hbonded_turn',
 'ss_prop_bend',
 'ss_prop_loop',
 'hydrophobic_fitness',
 'isoelectric_point',
 'rosetta_fa_rep',
 'rosetta_fa_intra_rep',
 'rosetta_lk_ball_wtd',
 'rosetta_fa_intra_sol_xover4',
 'rosetta_hbond_lr_bb',
 'rosetta_hbond_sr_bb',
 'rosetta_hbond_sc',
 'rosetta_pro_close',
 'aggrescan3d_avg_value',
 'aggrescan3d_max_value',]]
wholedata_rd.to_csv ('./core_database/prepared_dimension_reduction_data.csv', index = False)

Round 1 PCA

In [ ]:
wholedata = pd.read_csv('./core_database/destress_wholedata_labelled_as_rd.csv')
wholedata.columns=wholedata.columns.to_list()

features = [
 'ss_prop_hbonded_turn',
 'ss_prop_bend',
 'ss_prop_loop',
 'hydrophobic_fitness',
 'isoelectric_point',
 'rosetta_fa_rep',
 'rosetta_fa_intra_rep',
 'rosetta_lk_ball_wtd',
 'rosetta_fa_intra_sol_xover4',
 'rosetta_hbond_lr_bb',
 'rosetta_hbond_sr_bb',
 'rosetta_hbond_sc',
 'rosetta_pro_close',
 'aggrescan3d_avg_value',
 'aggrescan3d_max_value']

In [ ]:
#Round 1 PCA
# Separating out the 
x_1 = wholedata.loc[:, features].values

# Separating out the target
y_1 = wholedata.loc[:,['Entry', 'label_1', 'label_2','composition_hydrophobic', 'composition_polar', 'composition_acidic','composition_basic', 'composition_cysteine', 'composition_glycine','composition_proline']].values

# Standardizing the features
x_1 = MinMaxScaler().fit_transform(x_1)
pca_2d_1 = PCA(n_components=2)

principalComponents_1 = pca_2d_1.fit_transform(x_1)

principalDf_2d_1 = pd.DataFrame(data = principalComponents_1
             , columns = ['principal component 1', 'principal component 2'])


In [ ]:
#concatenate the data and label into one dataframe
z_1 =wholedata.loc[:,['label_1','label_2']]
z_1.reset_index(drop=True, inplace=True)
finalDf_2d_1 = pd.concat([principalDf_2d_1, z_1], axis = 1)
#finalDf_2d_1 =finalDf_2d_1.sample(n = 10000, random_state=0)

In [ ]:

# Use the PCA data from your previous code
af2_data = finalDf_2d_1[(finalDf_2d_1['label_1'] == 'AF2') | (finalDf_2d_1['label_2'] == 'AF2')]
pdb_data = finalDf_2d_1[(finalDf_2d_1['label_1'] == 'PDB') | (finalDf_2d_1['label_2'] == 'PDB')]
pisces_data = finalDf_2d_1[(finalDf_2d_1['label_1'] == 'Pisces') | (finalDf_2d_1['label_2'] == 'Pisces')]

x_af2 = af2_data['principal component 1'].values
y_af2 = af2_data['principal component 2'].values
x_pdb = pdb_data['principal component 1'].values
y_pdb = pdb_data['principal component 2'].values
x_pisces = pisces_data['principal component 1'].values
y_pisces = pisces_data['principal component 2'].values


def scatter_hist(x, y, ax, ax_histx, ax_histy, color):
    ax.tick_params(axis='x', which='major', labelsize=20)  # Change 20 to your desired font size for x-axis ticks
    ax.tick_params(axis='y', which='major', labelsize=20)  # Change 20 to your desired font size for y-axis ticks
   # no labels
    ax_histx.tick_params(axis="x", labelbottom=False)
    ax_histy.tick_params(axis="y", labelleft=False)

    # the scatter plot:
    ax.scatter(x, y, c=color, alpha=0.5)
    
    # now determine nice limits by hand:
    binwidth = 0.25

    bins_x = np.linspace(min(x), max(x), 200)
    bins_y = np.linspace(min(y), max(y),200)

    ax_histx.tick_params(axis='x', which='major', labelbottom=False)
    ax_histx.tick_params(axis='y', which='major',labelsize=20)
    
    ax_histy.tick_params(axis='y', which='major', labelleft=False)
    ax_histy.tick_params(axis='x', which='major', labelsize=20,rotation =270)
    
    ax_histx.hist(x, bins=bins_x, color=color, alpha=0.7, edgecolor='none')
    ax_histy.hist(y, bins=bins_y, orientation='horizontal', color=color, alpha=0.7, edgecolor='none')


def add_loading_vectors(ax, loadings, labels, scale=1, fontsize=25, color='r', threshold=0.15, label_offset_ratio=0.07):
    for i, (x, y) in enumerate(loadings.T):
        if abs(x) >= threshold or abs(y) >= threshold:
            ax.arrow(0, 0, x * scale, y * scale, color=color, alpha=0.5)
            # Calculate the label position based on the direction of the loading vector
            label_x = x * scale * (1 + label_offset_ratio)
            label_y = y * scale * (1 + label_offset_ratio)
            ax.text(label_x, label_y, labels[i], color=color, fontsize=fontsize, ha='center', va='center')

loadings = pca_2d_1.components_

In [ ]:
# Start with a square Figure.
fig = plt.figure(figsize=(25,25))

# Add a gridspec with two rows and two columns and a ratio of 1 to 4 between
# the size of the marginal axes and the main axes in both directions.
# Also adjust the subplot parameters for a square plot.
gs = fig.add_gridspec(2, 2,  width_ratios=(5, 1), height_ratios=(1, 5),
                      left=0.1, right=0.9, bottom=0.1, top=0.9,
                      wspace=0.05, hspace=0.05)

# Create the Axes.
ax = fig.add_subplot(gs[1, 0])
ax_histx = fig.add_subplot(gs[0, 0], sharex=ax)
ax_histy = fig.add_subplot(gs[1, 1], sharey=ax)

xticks = np.arange(-1, 1.2, 0.2)
yticks = np.arange(-1, 1.2, 0.2)
ax.set_xticks(xticks)
ax.set_yticks(yticks)
targets = [ 'AF2','PDB','Pisces',]
#targets = [ 'PDB','AF2']

colors = [ '#D5D6BF','#B87C1B', '#6279A2',]
#colors = ['#B87C1B', '#D5D6BF']

# Set tick label size for the scatter plot
ax.tick_params(axis='both', which='major', labelsize=20)

# Draw the scatter plot and marginals.
scatter_hist(x_af2, y_af2, ax, ax_histx, ax_histy, color='#D5D6BF')
scatter_hist(x_af2, y_af2, ax, ax_histx, ax_histy, color='#D5D6BF')
scatter_hist(x_pdb, y_pdb, ax, ax_histx, ax_histy, color='#B87C1B')
scatter_hist(x_pisces, y_pisces, ax, ax_histx, ax_histy, color='#6279A2')

# Add the loading vectors to the plot
labels = features # Replace with your original variable names
add_loading_vectors(ax, loadings, labels, fontsize=25, color='red')

# Set axis labels

ax.set_xlabel('Principal Component 1', fontsize = 30)
ax.set_ylabel('Principal Component 2', fontsize = 30)
fig.suptitle('2 component PCA of DE-STRESS database', fontsize = 50, y=0.93)

# Create legend
legend_handles = [Patch(facecolor=color, edgecolor=color, label=str(target),
                        linewidth=1, alpha=0.8) for target, color in zip(targets, colors)]

ax.legend(handles=legend_handles, prop={'size': 45})
plt.savefig('./figures/dimension_reduction/PCA_.png',dpi=300)
# Display the plot
plt.show()


In [ ]:
#visualize the PCA result on scatter plot

fig = plt.figure(figsize = (20,20))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 20)
ax.set_ylabel('Principal Component 2', fontsize = 20)
ax.set_title('2 component PCA of DE-STRESS database', fontsize = 35)
ax.set_xticks(np.arange(-1, 1, 0.1))
ax.set_yticks(np.arange(-1, 1, 0.1))

targets = [ 'AF2','PDB','Pisces',]
#targets = [ 'PDB','AF2']

colors = [ '#D5D6BF','#B87C1B', '#6279A2',]
#colors = ['#B87C1B', '#D5D6BF']

for target, color in zip(targets,colors):
    indicesToKeep = (finalDf_2d_1['label_1'] == target) | (finalDf_2d_1['label_2'] == target)
    ax.scatter(finalDf_2d_1.loc[indicesToKeep, 'principal component 1']
               , finalDf_2d_1.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50
               , alpha=0.5)
    
legend_handles = [Patch(facecolor=color, edgecolor=color, label=str(target),
                        linewidth=1, alpha=0.8) for target, color in zip(targets, colors)]

# Add the custom legend to the plot

ax.legend(handles=legend_handles, prop={'size': 30})
regions = [
    
    {'center': (0, 0.7), 'radius':  0.05,'label': 1},
    {'center': (0, -0.17), 'radius':  0.05,'label': 3},
    {'center': (0, -0.55), 'radius':  0.05,'label': 4},
    {'center': (-0.55, 0), 'radius':  0.05,'label': 2},
    {'center': (0.1, 0), 'radius':  0.05,'label': 5},
    {'center': (0.4, 0), 'radius':  0.05,   'label': 6},
    {'center': (0.72, 0), 'radius':  0.05,   'label': 7},
]
for region in regions:
    circle = Circle(region['center'], region['radius'], edgecolor='red',
                    facecolor='none', linewidth=2)
    ax.add_patch(circle)
    label_x = region['center'][0] + region['radius'] + 0.01  # Adjust the '5' for the desired offset
    label_y = region['center'][1]
    
    # For label '4', adjust the y position to be above the circle
    if region['label'] == 2:
        label_y += region['radius'] + 0.02
    
    # Increase fontsize and set weight to 'bold' for thicker text
    ax.annotate(str(region['label']), (label_x, label_y), fontsize=40, weight='bold', ha='left', va='center')
ax.grid()
# Display the plot
plt.savefig('./figures/dimension_reduction/PCA_for_region_labeling.png',dpi=300)
fig.show()


In [ ]:
def hex_to_rgb(hex_color):
    hex_color = hex_color.lstrip('#')
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))
hex_color = '#6279A2' #[ '#D5D6BF','#B87C1B', '#6279A2',]

rgb_color = hex_to_rgb(hex_color)
print(rgb_color)


In [ ]:
#Round 1 PCA: To find out the ratio of explained variance by each components
#PCA result analysis
explained_variance_ratio = pca_2d_1.explained_variance_ratio_
print("Explained Variance Ratio:", explained_variance_ratio)


In [ ]:

# Step 3: Interpret principal components
loadings = pca_2d_1.components_
loadings
loading_matrix = pd.DataFrame(loadings, columns=features)
#print("Loading Matrix:\n", loading_matrix)
loading_matrix.to_csv('./output_tables/2dPCA_1_principal_components_interpretation_1.csv')


Find and remove the outliers

In [ ]:
#outlier finder function
def find_extreme_outliers(df, col1, col2):
    # Calculate IQR for both columns
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1
    
    # Define Boolean masks for extreme outliers in each column
    mask1 = (df[col1] < Q1[col1] - 1.5 * IQR[col1]) | (df[col1] > Q3[col1] + 1.5 * IQR[col1])
    mask2 = (df[col2] < Q1[col2] - 1.5 * IQR[col2]) | (df[col2] > Q3[col2] + 1.5 * IQR[col2])
    
    # Combine masks to find rows with extreme outliers in either column
    outliers = df[(mask1 | mask2)]
    
    return outliers

In [ ]:
outliers = find_extreme_outliers(principalDf_2d_1, "principal component 1", "principal component 2")
outlier_list = outliers.index.to_list()

In [ ]:
#visualize the PCA result on scatter plot
fig = plt.figure(figsize = (25,25))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 20)
ax.set_ylabel('Principal Component 2', fontsize = 20)
ax.set_title('2 component PCA of DE-STRESS database', fontsize = 35)

targets = [ 'AF2','PDB','Pisces','Outliers']
#targets = [ 'PDB','AF2']

colors = [ '#D5D6BF','#B87C1B', '#6279A2','#E04C5E']
#colors = ['#B87C1B', '#D5D6BF']

for target, color in zip(targets,colors):
    indicesToKeep = (finalDf_2d_1['label_1'] == target) | (finalDf_2d_1['label_2'] == target)
    ax.scatter(finalDf_2d_1.loc[indicesToKeep, 'principal component 1']
               , finalDf_2d_1.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50
               , alpha=0.5)
    # Outliers
    outlier_indices = indicesToKeep & finalDf_2d_1.index.isin(outlier_list)
    ax.scatter(finalDf_2d_1.loc[outlier_indices, 'principal component 1']
               , finalDf_2d_1.loc[outlier_indices, 'principal component 2']
               , c = '#E04C5E'
               , s = 50
               , alpha=0.5
               )  # Square marker
legend_handles = [Patch(facecolor=color, edgecolor=color, label=str(target), linewidth=1, alpha=0.8) for target, color in zip(targets, colors)]

# Add the custom legend to the plot
ax.legend(handles=legend_handles, prop={'size': 30})

ax.grid()


find retions


In [ ]:
def retrieve_entry_values(outliers, wholedata):
    # Get the index values of the outliers
    outlier_indices = outliers.index
    
    # Retrieve the corresponding Entry values from the wholedata DataFrame
    entry_values = wholedata.loc[outlier_indices, 'Entry']
    
    # Return the Entry values as a DataFrame
    return entry_values.to_frame()

#find interesting area in the plot 
def areafinder(df, min_1, max_1, min_2, max_2):
    list_of_structures = []
    for index, row in df.iterrows():
        if min_1 < row[0] < max_1 and min_2 < row[1] < max_2:
            list_of_structures.append(index)

    list_of_uniprot_ids = []
    for item in list_of_structures:
        uniprot_id = uniprotidfinder(item)
        list_of_uniprot_ids.append(uniprot_id)

    return list_of_uniprot_ids
def uniprotidfinder(index):
    return wholedata.loc[index, ['Entry','label_2']]


In [ ]:
region1 = areafinder(finalDf_2d_1, -0.05, 0.05, 0.65, 0.75)
region2 = areafinder(finalDf_2d_1, -0.05, 0.05, -0.22, -0.12)
region3 = areafinder(finalDf_2d_1, -0.05, 0.05, -0.6, -0.5)
region4 = areafinder(finalDf_2d_1, -0.6, -0.5, -0.05, 0.05)
region5 = areafinder(finalDf_2d_1, 0.05, 0.15, -0.05, 0.05)
region6 = areafinder(finalDf_2d_1, 0.35, 0.45, -0.05, 0.05)
region7 = areafinder(finalDf_2d_1, 0.67, 0.77, -0.05, 0.05)

In [ ]:
region7


In [ ]:
import random
num_samples = 100
random_elements = random.sample(region5, num_samples)

print(random_elements)

In [ ]:
entry_value = "6B87"
selected_rows = entry_values_outliers.loc[entry_values_outliers['Entry'] == entry_value]
selected_rows

In [ ]:
finalDf_2d_1.iloc[1067]

In [ ]:
wholedata['composition_glycine'].describe()
#'charge', 'mass','number of residues','composition_hydrophobic','composition_polar','composition_acidic', 
#'composition_basic','composition_cysteine','composition_glycine', 'composition_proline','label_3'


#'isoelectric_point','mass',
# 'composition_hydrophobic',
# 'composition_acidic','composition_basic','composition_cysteine','composition_glycine',
# 'composition_proline','label_3']

In [ ]:
 'hydrophobic_fitness',
 'packing_density',
 'composition_hydrophobic',
 'label_3'

In [ ]:
wholedata['packing_density'].describe()

Label the PCA results by composition of glycine

In [ ]:
# Define the bins and labels
bins = [float('-inf'), 0.047,0.081, float('inf')]
labels = ['less gly','moderate gly','more gly']
wholedata_1=wholedata.copy()
# Categorize the data
wholedata_1['category'] = pd.cut(wholedata['composition_glycine'], bins=bins, labels=labels)

#concatenate the data and label into one dataframe
z_1 =wholedata_1.loc[:,['category']]
z_1.reset_index(drop=True, inplace=True)
finalDf_2d_1 = pd.concat([principalDf_2d_1, z_1], axis = 1)
finalDf_2d_1 =finalDf_2d_1.sample(n = 100000, random_state=0)

In [ ]:
#visualize the PCA result on scatter plot
fig = plt.figure(figsize = (35,35))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)

targets = ['less gly','moderate gly','more gly']
colors = [ '#D73027','#FFFFBF','#ABD9E9']
for target, color in zip(targets,colors):
    indicesToKeep = (finalDf_2d_1['category'] == target)
    ax.scatter(finalDf_2d_1.loc[indicesToKeep, 'principal component 1']
               , finalDf_2d_1.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50
               , alpha=0.5)
ax.legend(targets,prop={'size': 30})
ax.grid()


Label the PCA results by hydrophobic fitness

In [ ]:
# Define the bins and labels
bins = [float('-inf'), 0.18,0.24, float('inf')]
labels = ['less polar','moderate polar','more polar']
wholedata_1=wholedata.copy()
# Categorize the data
wholedata_1['category'] = pd.cut(wholedata['composition_polar'], bins=bins, labels=labels)

#concatenate the data and label into one dataframe
z_1 =wholedata_1.loc[:,['category']]
z_1.reset_index(drop=True, inplace=True)
finalDf_2d_1 = pd.concat([principalDf_2d_1, z_1], axis = 1)
finalDf_2d_1 =finalDf_2d_1.sample(n = 10000, random_state=0)

In [ ]:
#visualize the PCA result on scatter plot
fig = plt.figure(figsize = (35,35))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)

targets = ['less polar','moderate polar','more polar']
colors = [ '#D73027','#FFFFBF','#ABD9E9']
for target, color in zip(targets,colors):
    indicesToKeep = (finalDf_2d_1['category'] == target)
    ax.scatter(finalDf_2d_1.loc[indicesToKeep, 'principal component 1']
               , finalDf_2d_1.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50
               , alpha=0.5)
ax.legend(targets,prop={'size': 30})
ax.grid()
fig.show()

Label the PCA results by charge

In [ ]:
# Define the bins and labels
bins = [float('-inf'), 0, float('inf')]
labels = ['negatively charged','positively charged']
wholedata_1=wholedata.copy()
# Categorize the data
wholedata_1['category'] = pd.cut(wholedata['charge'], bins=bins, labels=labels)

#concatenate the data and label into one dataframe
z_1 =wholedata_1.loc[:,['category']]
z_1.reset_index(drop=True, inplace=True)
finalDf_2d_1 = pd.concat([principalDf_2d_1, z_1], axis = 1)
finalDf_2d_1 =finalDf_2d_1.sample(n = 10000, random_state=0)

In [ ]:
#visualize the PCA result on scatter plot
fig = plt.figure(figsize = (35,35))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)

targets = [ 'negatively charged','positively charged']
colors = [ '#D73027','#ABD9E9']
for target, color in zip(targets,colors):
    indicesToKeep = (finalDf_2d_1['category'] == target)
    ax.scatter(finalDf_2d_1.loc[indicesToKeep, 'principal component 1']
               , finalDf_2d_1.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50
               , alpha=0.5)
ax.legend(targets,prop={'size': 30})
ax.grid()


Label the PCA results by mass

In [ ]:
# Define the bins and labels
bins = [float('-inf'), 2.185384e+04,6.523271e+04, float('inf')]
labels = ['small','middle','large']
wholedata_1  = wholedata.copy()
# Categorize the data
wholedata_1['category'] = pd.cut(wholedata['mass'], bins=bins, labels=labels)

#concatenate the data and label into one dataframe
z_1 =wholedata_1.loc[:,['category']]
z_1.reset_index(drop=True, inplace=True)
finalDf_2d_1 = pd.concat([principalDf_2d_1, z_1], axis = 1)
finalDf_2d_1 =finalDf_2d_1.sample(n = 10000, random_state=0)

In [ ]:

#visualize the PCA result on scatter plot
fig = plt.figure(figsize = (35,35))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 20)
ax.set_ylabel('Principal Component 2', fontsize = 20)
ax.set_title('2 component PCA labeled with mass', fontsize = 35)

targets = [ 'small','middle','large']
colors = [  '#D73027','#FFFFBF','#ABD9E9',]
for target, color in zip(targets,colors):
    indicesToKeep = (finalDf_2d_1['category'] == target)
    ax.scatter(finalDf_2d_1.loc[indicesToKeep, 'principal component 1']
               , finalDf_2d_1.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50
               , alpha=1)
legend_handles = [Patch(facecolor=color, edgecolor=color, label=str(target), linewidth=1, alpha=0.8) for target, color in zip(targets, colors)]

ax.legend(handles=legend_handles, prop={'size': 30})

ax.grid()
#plt.savefig('./figures/dimension_reduction/PCA_mass.png',dpi=300)
plt.show()

Label the PCA results by the percentage of hydrophobic residues

In [ ]:
# Define the bins and labels
bins = [float('-inf'), 0.36, 0.43, float('inf')]
labels = ['<36%','36-43%','43%']
wholedata_1  = wholedata.copy()
# Categorize the data
wholedata_1['category'] = pd.cut(wholedata['composition_hydrophobic'], bins=bins, labels=labels)

#concatenate the data and label into one dataframe
z_1 =wholedata_1.loc[:,['category']]
z_1.reset_index(drop=True, inplace=True)
finalDf_2d_1 = pd.concat([principalDf_2d_1, z_1], axis = 1)
finalDf_2d_1 =finalDf_2d_1.sample(n = 100000, random_state=0)

In [ ]:

#visualize the PCA result on scatter plot
fig = plt.figure(figsize = (30,30))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 20)
ax.set_ylabel('Principal Component 2', fontsize = 20)
ax.set_title('2 component PCA labeled with hydrophobic residue percentage', fontsize = 35)

targets = [ '<36%','36-43%','43%']
colors = [  '#D73027','#FFFFBF','#ABD9E9',]
for target, color in zip(targets,colors):
    indicesToKeep = (finalDf_2d_1['category'] == target)
    ax.scatter(finalDf_2d_1.loc[indicesToKeep, 'principal component 1']
               , finalDf_2d_1.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50
               , alpha=0.2)
legend_handles = [Patch(facecolor=color, edgecolor=color, label=str(target), linewidth=1, alpha=0.8) for target, color in zip(targets, colors)]

ax.legend(handles=legend_handles, prop={'size': 30})

ax.grid()
#plt.savefig('./figures/dimension_reduction/PCA_composition_hydrophobic.png',dpi=300)
plt.show()

Label the PCA results by the percentage of proline residues

In [ ]:
# Define the bins and labels
bins = [float('-inf'), 0.035, 0.062, float('inf')]
labels = ['<3.5%','3.5-6.2%','6.2%']
wholedata_1  = wholedata.copy()
# Categorize the data
wholedata_1['category'] = pd.cut(wholedata['composition_proline'], bins=bins, labels=labels)

#concatenate the data and label into one dataframe
z_1 =wholedata_1.loc[:,['category']]
z_1.reset_index(drop=True, inplace=True)
finalDf_2d_1 = pd.concat([principalDf_2d_1, z_1], axis = 1)

#visualize the PCA result on scatter plot
fig = plt.figure(figsize = (30,30))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 20)
ax.set_ylabel('Principal Component 2', fontsize = 20)
ax.set_title('2 component PCA labeled with proline residue percentage', fontsize = 35)

targets = ['<3.5%','3.5-6.2%','6.2%']
colors = [  '#D73027','#FFFFBF','#ABD9E9',]
for target, color in zip(targets,colors):
    indicesToKeep = (finalDf_2d_1['category'] == target)
    ax.scatter(finalDf_2d_1.loc[indicesToKeep, 'principal component 1']
               , finalDf_2d_1.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50
               , alpha=0.5)
legend_handles = [Patch(facecolor=color, edgecolor=color, label=str(target), linewidth=1, alpha=0.8) for target, color in zip(targets, colors)]

ax.legend(handles=legend_handles, prop={'size': 30})

ax.grid()
plt.savefig('./figures/dimension_reduction/PCA_composition_proline.png',dpi=300)
plt.show()

Label the PCA results by the percentage of charged residues

In [ ]:
# Define the bins and labels
bins = [float('-inf'), 0.116, 0.157, float('inf')]
labels = ['<11.6%','11.6-15.7%','15.7%']
wholedata_1  = wholedata.copy()
# Categorize the data
wholedata_1['category'] = pd.cut(wholedata['composition_basic'], bins=bins, labels=labels)

#concatenate the data and label into one dataframe
z_1 =wholedata_1.loc[:,['category']]
z_1.reset_index(drop=True, inplace=True)
finalDf_2d_1 = pd.concat([principalDf_2d_1, z_1], axis = 1)

#visualize the PCA result on scatter plot
fig = plt.figure(figsize = (30,30))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 20)
ax.set_ylabel('Principal Component 2', fontsize = 20)
ax.set_title('2 component PCA labeled with proline residue percentage', fontsize = 35)

targets = ['<3.5%','3.5-6.2%','6.2%']
colors = [  '#D73027','#FFFFBF','#ABD9E9',]
for target, color in zip(targets,colors):
    indicesToKeep = (finalDf_2d_1['category'] == target)
    ax.scatter(finalDf_2d_1.loc[indicesToKeep, 'principal component 1']
               , finalDf_2d_1.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50
               , alpha=0.5)
legend_handles = [Patch(facecolor=color, edgecolor=color, label=str(target), linewidth=1, alpha=0.8) for target, color in zip(targets, colors)]

ax.legend(handles=legend_handles, prop={'size': 30})

ax.grid()
plt.savefig('./figures/dimension_reduction/PCA_composition_proline.png',dpi=300)
plt.show()

Label the PCA results by the Category from CATH classification

In [ ]:
wholedata_2 = wholedata.copy()
wholedata_2['label_4'] = wholedata_2['label_3'].str[:1]
wholedata_2

In [ ]:
wholedata_2 = wholedata.copy()
wholedata_2['label_4'] = wholedata_2['label_3'].str[:1]

principalDf_2d_1 = pd.DataFrame(data = principalComponents_1
             , columns = ['principal component 1', 'principal component 2'])

#concatenate the data and label into one dataframe
z =wholedata_2.loc[:,['label_4']]
z.reset_index(drop=True, inplace=True)

finalDf = pd.concat([principalDf_2d_1, z], axis = 1)
finalDf.dropna(inplace=True)
finalDf.reset_index(drop=True, inplace=True)

In [ ]:

# Assuming you have the finalDf DataFrame with the 'label_4' column

fig = plt.figure(figsize=(35, 35))
ax = fig.add_subplot(1, 1, 1)

ax.set_xlabel('Principal Component 1', fontsize=20)
ax.set_ylabel('Principal Component 2', fontsize=20)
ax.set_title('PCA analysis result labeled with structural category', fontsize=35)
#ax.set_xticks(np.arange(-120, 120, 5))
#ax.set_yticks(np.arange(-120, 120, 5))

targets = ['1','2','3','4','6']#[1.0, 2.0, 3.0, 4.0, 5.0, 6.0]
colors = ['#4575b4',  '#fee090', '#e0f3f8', '#fc8d59', '#d73027']
for target, color in zip(targets, colors):
    indicesToKeep = finalDf['label_4'] == target
    ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1'],
               finalDf.loc[indicesToKeep, 'principal component 2'],
               c=color,
               s=5,
               alpha=0.8)


ax.grid()
# Create custom legend handles
legend_handles = [Patch(facecolor=color, edgecolor=color, label=str(target), linewidth=1, alpha=0.8) for target, color in zip(targets, colors)]
ax.legend(handles=legend_handles, prop={'size': 30})
#plt.savefig('./figures/dimension_reduction/PCA_CATH_labeling.png',dpi=300)
# Add the custom legend to the plot

Round 2 PCA

In [ ]:
#Round 2 PCA
#remove the extreme outliers and conduct pca again
#reindexing pdb and drop the identified extreme outliers 
wholedata_2 = wholedata.copy()
wholedata_2.reset_index(drop=True, inplace = True)
wholedata_2.drop(outlier_list ,inplace = True)

# Separating out the features
x_2 = wholedata_2.loc[:, features].values

# Separating out the target
y_2 = wholedata_2.loc[:,['label_1','label_2']].values

# Standardizing the features
x_2 = MinMaxScaler().fit_transform(x_2)
pca_2d_2 = PCA(n_components=2)

principalComponents_2d_2= pca_2d_2.fit_transform(x_2)

principalDf_2d_2 = pd.DataFrame(data = principalComponents_2d_2
             , columns = ['principal component 1', 'principal component 2'])


In [ ]:
finalDf_2d_2

In [ ]:

z =wholedata_2.loc[:,['label_1','label_2']]
z.reset_index(drop=True, inplace=True)
finalDf_2d_2 = pd.concat([principalDf_2d_2, z], axis = 1)


fig = plt.figure(figsize = (35,35))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA_2', fontsize = 20)

targets = [ 'AF2','PDB', 'Pisces']
colors = [ '#D5D6BF', '#B87C1B','#6279A2']
for target, color in zip(targets,colors):
    indicesToKeep = (finalDf_2d_2['label_1'] == target) | (finalDf_2d_2['label_2'] == target)
    ax.scatter(finalDf_2d_2.loc[indicesToKeep, 'principal component 1']
               , finalDf_2d_2.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50
               , alpha=0.8)
ax.legend(targets,prop={'size': 30})
ax.grid()

In [ ]:
#Round 2 PCA: To find out the ratio of explained variance by each components
#PCA result analysis
explained_variance_ratio = pca_2d_2.explained_variance_ratio_
print("Explained Variance Ratio:", explained_variance_ratio)

# Step 3: Interpret principal components
loadings = pca_2d_2.components_

loading_matrix = pd.DataFrame(loadings, columns=features)
#print("Loading Matrix:\n", loading_matrix)
loading_matrix.to_csv('./output_tables/2dPCA_1_principal_components_interpretation_2.csv')


In [ ]:
finalDf_2d_1.describe()
-6.65e-01	8.18e-01
-7.01-01    7.60-01

In [ ]:
finalDf_2d_2.describe()
-7.05e-01	7.78e-01
-4.75e-01   7.22e-01

In [ ]:
wholedata_new = wholedata.copy()


In [ ]:
wholedata_new.to_csv('wholedata_pdb_af2.csv')

Round 3 PCA

In [ ]:
#find all outliers
#find out all outliers in the 2dpca_1
principalDf_2d_1.columns=['principal component 1', 'principal component 2']
principalDf_2d_1_3 = principalDf_2d_1.copy()

X_3 = principalDf_2d_1_3.iloc[:, 0:2].values


pc1_upper = 0.53 
pc1_lower = -0.59
pc1_all_outliers = (X_1[:, 0] > pc1_upper) | (X_1[:, 0] < pc1_lower)

pc2_upper = 0.52 
pc2_lower = -0.52 
pc2_all_outliers = (X_1[:, 1] > pc2_upper) | (X_1[:, 1] < pc2_lower)

all_outliers = pd.concat([principalDf_2d_1_3[pc1_all_outliers], principalDf_2d_1_3[pc2_all_outliers]], axis=0)
all_outliers_list = all_outliers.index.tolist()
len (all_outliers_list)

In [ ]:
#Round 3 PCA
#remove the all outliers and conduct pca again
wholedata_3 = wholedata_1.copy()
wholedata_3.reset_index(drop=True, inplace=True)
wholedata_3.drop(all_outliers_list,inplace=True) #pdb_3 = pdb with removed extreme outliers


# Separating out the features
x_3 = wholedata_3.loc[:, features].values

# Separating out the target
y_3 = wholedata_3.loc[:,['label_1','label_2']].values

# Standardizing the features
x_3 = MinMaxScaler().fit_transform(x_3)
pca_2d_3 = PCA(n_components=2)

principalComponents_2d_3= pca_2d_3.fit_transform(x_3)

principalDf_2d_3 = pd.DataFrame(data = principalComponents_2d_3
             , columns = ['principal component 1', 'principal component 2'])

z_3 =wholedata_3.loc[:,['label_1','label_2']]
z_3.reset_index(drop=True, inplace=True)
finalDf_2d_3 = pd.concat([principalDf_2d_3, z_3], axis = 1)


fig = plt.figure(figsize = (35,35))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA_3', fontsize = 20)

targets = ['AF2','PDB',  'Pisces']
colors = ['#D5D6BF','#B87C1B',  '#6279A2']
for target, color in zip(targets,colors):
    indicesToKeep = (finalDf_2d_3['label_1'] == target) | (finalDf_2d_3['label_2'] == target)
    ax.scatter(finalDf_2d_3.loc[indicesToKeep, 'principal component 1']
               , finalDf_2d_3.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50
               , alpha=0.8)
ax.legend(targets,prop={'size': 30})
ax.grid()

In [ ]:
#PCA result analysis
explained_variance_ratio = pca_2d_3.explained_variance_ratio_
print("Explained Variance Ratio:", explained_variance_ratio)

# Step 3: Interpret principal components
features = [
       'hydrophobic_fitness', 'isoelectric_point', 'charge',
       'packing_density',  'evoef2_total',
       'evoef2_intraR_total', 'rosetta_total', 'rosetta_fa_intra_rep',
       'rosetta_fa_elec', 'rosetta_fa_sol', 'rosetta_lk_ball_wtd',
       'rosetta_fa_intra_sol_xover4', 'rosetta_hbond_bb_sc',
       'rosetta_hbond_sc', 'rosetta_rama_prepro', 'rosetta_p_aa_pp',
       'rosetta_fa_dun', 'rosetta_omega', 'rosetta_pro_close',
       'aggrescan3d_avg_value', 'aggrescan3d_max_value']
loadings = pca_2d_3.components_
loadings
#loading_matrix = pd.DataFrame(loadings, columns=features)
#print("Loading Matrix:\n", loading_matrix)
loading_matrix.to_csv('2dPCA_3_principal_components_interpretation.csv')